In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from glob import glob
import csv
import yaml
from chatcaptioner.chat import get_chat_log
from chatcaptioner.blip2 import Blip2
from chatcaptioner.utils import print_info, plot_img, extractQA_chatgpt, RandomSampledDataset

In [ ]:
# specify SAVE_PATH to visualize the result you want
SAVE_PATH = 'experiments/test/'
DATA_ROOT = 'datasets/'

In [ ]:
datasets_list = os.listdir(SAVE_PATH)
datasets_list = ['cc_val']
yes_no_list = []

for dataset_name in datasets_list:
    print('============================')
    print('          {}          '.format(dataset_name))
    print('============================')
    dataset = RandomSampledDataset(DATA_ROOT, dataset_name)
    
    save_infos = glob(os.path.join(SAVE_PATH, dataset_name, 'caption_result', '*'))
    for info_file in save_infos:
        with open(info_file, 'r') as f:
            info = yaml.safe_load(f)
        img_id = info['id'] if 'id' in info else info['setting']['id']
        test_img, _ = dataset.fetch_img(img_id)
        
        chat = info['FlanT5 XXL']['ChatCaptioner']['chat']
        questions, answers = extractQA_chatgpt(chat)
        yes_no = False
        for q, a in zip(questions, answers):
            a = a.lower()
            # a = a.split(' ')
            if 'yes' == a or 'no' == a:
            # if 'Are there' in q:
                yes_no = True
                print('Question: {}'.format(q))
                print('Answer: {}'.format(a))
                yes_no_list.append((img_id, q, a))
        if not_sure:
            plot_img(test_img)
        

In [ ]:
# save questions that BLIP-2 is unsure about for later human evalutaion
uncertainty_dict = {}
for img_id, q, a in uncertainty_list:
    if img_id not in uncertainty_dict:
        uncertainty_dict[img_id] = [q]
    else:
        uncertainty_dict[img_id].append(q)
with open(os.path.join('not_sure.yaml'), 'w') as f:
    yaml.dump(uncertainty_dict, f)